# Goal: map road section data to geometric representation in OSM data

## The data

In [5]:
df = pd.read_csv('PY2018-SECTIONS.csv')
df.columns = [c.strip().lower() for c in df.columns.to_list()]
df

,sect_id,sect_name,link_id,link_name,speed_flow,traf_flow,acc_class,road_class,clim_zone,surf_class,length,cway_width,shld_width,mt_aadt,nm_aadt,aadt_year,direction,rf,num_rfs,superelev,curvature,sigm_adral,speed_lim,enforcemnt,xnmt,xmt,xfri,hsnew,hsold,hbase,res_modulu,rel_compct,snp_derive,sn,cbr,snp_dry,d0,benkel_def,surf_stren,base_stren,subb_stren,hsubbase,surf_thick,slab_lenth,base_thick,base_modul,cnstr_year,subg_matrl,compmethod,cond_year,roughness,cracks_aca,cracks_acw,cracks_act,ravel_area,phole_num,edgebreak,rut_depth,rutdepth_sd,text_depth,skidresist,drain_cond,faulting,spall_jnts,crackslabs,detercrack,failureskm,grav_thick,last_const,last_surf,last_prvnt,last_rehab,prev_aca,prev_acw,prev_nct,lastgravel,drain_type,altitude,shouldtype,widn_width,edgedrains,nmt_separ,nmtlanes,elanes,calib_item,repcost,condbased,inirough,terrough,rdfosbgr_prreco,rdpvla_prreco,ftcycle_prreco,brdgstr_prreco,trfsgn_prreco,rdfosbgr_resval,rdpvla_resval,ftcycle_resval,brdgstr_resval,trfsgn_resval,rdfosbgr_uselfe,rdpvla_uselfe,ftcycle_uselfe,brdgstr_uselfe,trfsgn_uselfe,rdfosbgr_age,rdpvla_age,ftcycle_age,brdgstr_age,trfsgn_age,compageyear,usflfeunit,id
0,01030401,ATT1E1 1° DE MARZO - CARAGUATAY,1.000304e+08,NaN,carretera de 2 carriles angostos,Inter-urbano,PY2018,Secundaria,Or,0,16.342791,6.5,0.0,0,0,2017,2,10,1,2,15,0.1,80,1.1,1,1,1,50.0,0,200,15,97,0,2.50,8.0,1,0.0,0,0.35,0.15,0.15,75,0,0,0,0,0,0,0,2018,7.0,5.0,1.5,1.5,10.0,0.0,10.0,5.0,1.5,0.7,0.45,2,0,0,0,0,0,0,2010,2010,2010,2010,0,0,0,0,7,173,0,0,0,0,0,2,TSBG,0,1,2,12,0,100,0,0,0,0,0,0,0,0,10000,0,0,0,0,0,0,0,0,0,2012,3,1
1,01060102,ATT1E1 ABAI - CAPIITENDY - TARUMA - TUNA,4.000000e+00,NaN,carretera de 2 carriles angostos,Inter-urbano,PY2018,Terciaria,Or,0,1.193982,6.5,0.0,0,0,2017,2,10,1,2,15,0.1,80,1.1,1,1,1,50.0,0,200,15,97,0,2.62,8.0,1,0.0,0,0.35,0.15,0.15,75,0,0,0,0,0,0,0,2018,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5,0.60,2,0,0,0,0,0,0,2018,2018,2018,2018,0,0,0,0,7,232,0,0,0,0,0,2,TSBG,0,1,2,12,0,100,0,0,0,0,0,0,0,0,10000,0,0,0,0,0,0,0,0,0,2012,3,2
2,01071601,ATT1E1 ACCESO A ALTO VERA,7.000000e+00,NaN,carretera de 2 carriles angostos,Inter-urbano,PY2018,Terciaria,Or,0,10.467350,6.1,1.5,0,0,2017,2,10,1,2,15,0.1,80,1.1,1,1,1,50.0,0,200,15,97,0,2.62,8.0,1,0.0,0,0.35,0.15,0.15,75,0,0,0,0,0,0,0,2018,5.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.7,0.45,2,0,0,0,0,0,0,2015,2015,2015,2015,0,0,0,0,7,233,0,0,0,0,0,2,TSBG,0,1,2,12,0,100,0,0,0,0,0,0,0,0,10000,0,0,0,0,0,0,0,0,0,2012,3,3
3,01120103,ATT1E1 ACCESO A ISLA UMBU,9.000000e+00,NaN,carretera de 2 carriles angostos,Inter-urbano,PY2018,Secundaria,Or,0,1.219874,6.5,0.0,0,0,2017,2,10,1,2,15,0.1,80,1.1,1,1,1,50.0,0,200,15,97,0,2.62,8.0,1,0.0,0,0.35,0.15,0.15,75,0,0,0,0,0,0,0,2018,5.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.7,0.45,2,0,0,0,0,0,0,2016,2016,2016,2016,0,0,0,0,7,64,0,0,0,0,0,2,TSBG,0,1,2,12,0,100,0,0,0,0,0,0,0,0,10000,0,0,0,0,0,0,0,0,0,2012,3,4
4,01070401,ATT1E1 ACCESO A PUERTO TRIUNFO,1.200000e+01,NaN,carretera de 2 carriles angostos,Inter-urbano,PY2018,Secundaria,Or,0,14.659920,6.5,0.0,0,0,2017,2,10,1,2,15,0.1,80,1.1,1,1,1,50.0,0,200,15,97,0,2.62,8.0,1,0.0,0,0.35,0.15,0.15,75,0,0,0,0,0,0,0,2018,5.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.7,0.45,2,0,0,0,0,0,0,2016,2016,2016,2016,0,0,0,0,7,233,0,0,0,0,0,2,TSBG,0,1,2,12,0,100,0,0,0,0,0,0,0,0,10000,0,0,0,0,0,0,0,0,0,2012,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576,00000901,NT4E3 AVDA. ARTIGAS,NaN,NO ES COMPETENCIA DEL MOPC,carretera de 4 carriles,Urbano,PY2018,Primaria,Or,0,6.843380,14.0,0.0,0,0,2017,2,10,1,2,15,0.1,80,1.1,1,1,1,50.0,50,250,15,97,0,2.62,8.0,1,0.0,0,0.35,0.15,0.15,75,0,0,0,0,0,

#### Section data general scheme: `<place_name> - <place_name>`

#### But not always ...

Number of places if split on `-` for Guatemala data:
```
1 69
2 896
3 106
4 16
5 8
6 2
7 1
8 0
9 0
10 0
11 0
12 0
13 1
```

#### We also have a `length` field in the data.

---

## The approach, abstractly:

    road section data --> geometric features --> geometric representation in OSM data

#### For the 2 place cases, we can consider the 2 places to be the endpoints of the road.

#### And given the name of a place, we can try to find its geographical coordinates (**Geocoding**).

#### And given 2 points on a map, we can try to **find a route** between them using OSM road data.

# Algorithm walkthrough

## Trivial case

```python
def match(place_name_1, place_name_2):
    p1 = geocode(place_name_1)
    p2 = geocode(place_name_2)
    route = find_route(p1, p2)
    return route
```

---

## Realistic case: Each name may map to multiple possible locations

```python
def match(place_name_1, place_name_2, target_length):
    p1_candidates = geocode(place_name_1)
    p2_candidates = geocode(place_name_2)
    ...
```

### Example: Cruce Guarani -- Colonia Pindoty, 44.48 KM

![image showing candidate points for each endpoint](img/candidate_points.png)

---

```python
def match(place_name_1, place_name_2, target_length):
    p1_candidates = geocode(place_name_1)
    p2_candidates = geocode(place_name_2)
```
<pre><code>    <span style="display:inline-block; width:600px; background-color:#eee; font-size:1em">all_pairs = cartesian_product(p1_candidates, p2_candidates)</span>
    ...
</code></pre>

![image showing all pairs (connected with straight lines)](img/all_pairs.png)

---

### Performance optimization: Only consider feasible pairs pairs

```python
def match(place_name_1, place_name_2, target_length):
    p1_candidates = geocode(place_name_1)
    p2_candidates = geocode(place_name_2)
    
    all_pairs = cartesian_product(p1_candidates, p2_candidates)
```
<pre><code>    <span style="display:inline-block; width:600px; background-color:#eee; font-size:1em">candidate_pairs = select_candidate_pairs(all_pairs, target_length)</span>
    ...
</code></pre>

![image showing candidate pairs (connected with straight lines)](img/candidate_pairs.png)

---

```python
def match(place_name_1, place_name_2, target_length):
    p1_candidates = geocode(place_name_1)
    p2_candidates = geocode(place_name_2)

    all_pairs = cartesian_product(p1_candidates, p2_candidates)
    candidate_pairs = select_candidate_pairs(all_pairs, target_length)
```
<pre><code>    <span style="display:inline-block; width:600px; background-color:#eee; font-size:1em">candidate_routes = [find_route(p1, p2) for (p1, p2) in candidate_pairs]</span>
    ...
</code></pre>

![image candidate routes (connected with dashed curves)](img/candidate_routes.png)

---

In [ ]:
def match(place_name_1, place_name_2, target_length):
    p1_candidates = geocode(place_name_1)
    p2_candidates = geocode(place_name_2)

    all_pairs = cartesian_product(p1_candidates, p2_candidates)
    candidate_pairs = select_candidate_pairs(all_pairs, target_length)
    
    candidate_routes = [find_route(p1, p2) for (p1, p2) in candidate_pairs]
    best_route = select_best_route(candidate_routes, target_length)

    return best_route

```python
def match(place_name_1, place_name_2, target_length):
    p1_candidates = geocode(place_name_1)
    p2_candidates = geocode(place_name_2)

    all_pairs = cartesian_product(p1_candidates, p2_candidates)
    candidate_pairs = select_candidate_pairs(all_pairs, target_length)
    
    candidate_routes = [find_route(p1, p2) for (p1, p2) in candidate_pairs]
```
<pre><code>    <span style="display:inline-block; width:600px; background-color:#eee; font-size:1em">best_route = select_best_route(candidate_routes, target_length)</span></code></pre>
```python
    return best_route
```

![image best route among candidates](img/best_route.png)

---
---

# Results

## Guatemala

![](img/gt/length-scatter.png)

![](img/gt/diff-hist.png)

![](img/gt/diff-cdf.png)

---

## Paraguay

![](img/py/length-scatter.png)

![](img/py/diff-hist.png)

![](img/py/diff-cdf.png)

# Route finding

## Shortest path search in road network graph using `OSMnx` and `NetworkX`

```python
def find_route(p1, p2):
    start = closest_node(G, p1)
    end = closest_node(G, p2)
    route = shortest_path.dijkstra(G, start, end)
    return route
```

---

## Performance optimization: Use A* search instead of Dijkstra search

#### A* search is a kind of guided (by a heuristic) Dijkstra search.

```python
def find_route(p1, p2):
    start = closest_node(G, p1)
    end = closest_node(G, p2)
    route = shortest_path.a_star(G, start, end, heuristic=straight_line_distance)
    return route
```

---
---

# Geocoding

### Custom approach: fuzzy string matching on GeoNames dataset

### GeoNames data

In [14]:
places_df = load_geonames_data(file_paths['geonames'][COUNTRY])
places_df

,geonameid,name,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,cc2,...,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification,orig_name,geometry
0,3434138,isla santa isabel,Isla Santa Isabel,"isla entre rios,isla entre rios,isla santa isabel",-27.42043,-57.49094,T,ISL,PY,NaN,...,NaN,NaN,NaN,0,NaN,58,America/Asuncion,2017-10-04,Isla Santa Isabel,POINT (-57.49094 -27.42043)
1,3436529,24 de julio,24 De Julio,NaN,-25.08333,-57.30000,P,PPL,PY,NaN,...,306.0,NaN,NaN,0,NaN,71,America/Asuncion,2018-04-09,24 De Julio,POINT (-57.30000 -25.08333)
2,3436530,tres de mayo,Tres de Mayo,"3 de mayo,tres de mayo",-26.48134,-56.09592,P,PPL,PY,NaN,...,611.0,NaN,NaN,0,NaN,158,America/Asuncion,2020-06-10,Tres de Mayo,POINT (-56.09592 -26.48134)
3,3436531,fortin zenteno,Fortin Zenteno,"aliguata,alihuata,fortin zenteno,fortin zenten...",-23.16667,-59.98333,S,FT,PY,PY,...,1507.0,NaN,NaN,0,NaN,133,America/Asuncion,2020-06-10,Fortín Zenteno,POINT (-59.98333 -23.16667)
4,3436532,zeballos-cue,Zeballos-Cue,"zeballos-cue,zeballos-cue,zeballos-cue,zeballo...",-25.23019,-57.57118,P,PPLX,PY,NaN,...,0.0,NaN,NaN,0,NaN,78,America/Asuncion,2018-04-09,Zeballos-Cué,POINT (-57.57118 -25.23019)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18502,12174563,kokue guazu san antonio,Kokue Guazu San Antonio,"kokue guasu san antonio,kokue guazu san antoni...",-25.32743,-57.39412,P,PPL,PY,NaN,...,NaN,NaN,NaN,0,NaN,154,America/Asuncion,2020-07-13,Kokue Guazú San Antonio,POINT (-57.39412 -25.32743)
18503,12174826,parques del yacht,Parques del Yacht,parques del yacht,-25.36612,-57.63104,P,PPLX,PY,NaN,...,NaN,NaN,NaN,0,NaN,96,America/Asuncion,2020-07-13,Parques del Yacht,POINT (-57.63104 -25.36612)
18504,12174827,san juan,San Juan,san juan,-25.36281,-57.59567,P,PPLX,PY,NaN,...,NaN,NaN,NaN,0,NaN,120,America/Asuncion,2020-07-13,San Juan,POINT (-57.59567 -25.36281)
18505,12174828,isla san francisco,Isla San Francisco,isla san francisco,-25.13408,-57.52246,P,PPLX,PY,NaN,...,NaN,NaN,NaN,0,NaN,54,America/Asuncion,2020-07-13,Isla San Francisco,POINT (-57.52246 -25.13408)


### Fuzzy string matching

```
query: salto del guaira
[('salto del guaira', 100),
 ('saltos del guaira', 97),
 ('parque salto del guaira', 95),
 ('salto del guaira kilometro 18', 90),
 ('salto del guaira kilometro 32', 90)]
----------------------------
query: monte lindo
[('monte lindo', 100),
 ('zona monte lindo', 95),
 ('tapera monte lindo', 90),
 ('puesto monte lindo', 90),
 ('riacho monte lindo', 90)]
----------------------------
query: san joaqui
[('san joaquin', 95),
 ('serrania de san joaquin', 90),
 ('montes de san joaquin', 90),
 ('cerro san joaquin', 90),
 ('estancia san joaquin', 90)]
----------------------------
query: aldana canad
[('aldana canada', 96),
 ('asent. aldana canada', 90),
 ('asentamiento aldana canada', 90),
 ('ana', 90),
 ('aldama canada', 88)]
----------------------------
query: caraguata
[('caraguata', 100),
 ('caraguatay', 95),
 ('estero caraguatay-nu', 90),
 ('cerro caraguata', 90),
 ('arroyo caraguata', 90)]
----------------------------
query: cedrale
[('cedrales', 93),
 ('los cedrales', 90),
 ('colonia los cedrales', 90),
 ('catedral', 80),
 ('central', 71)]
----------------------------
```

---

## Use a geocoding API

Examples:  
- AlgoliaPlaces  
- ArcGIS  
- AzureMaps  
- Baidu  
- BaiduV3  
- BANFrance  
- Bing  
- DataBC  
- GeocodeEarth  
- GeocodeFarm  
- Geocodio  
- Geolake  
- GeoNames  
- GoogleV3  
- HERE  
- HEREv7  
- IGNFrance  
- MapBox  
- MapQuest  
- MapTiler  
- OpenCage  
- OpenMapQuest  
- Nominatim  
- Pelias  
- Photon  
- PickPoint  
- LiveAddress  
- TomTom  
- What3Words  
- What3WordsV3  
- Yandex  

---
---

# Results with geocoding via GeoNames API

## Guatemala

![](img/gt/geo/length-scatter.png)

![](img/gt/geo/diff-hist.png)

![](img/gt/geo/diff-cdf.png)

---

## Paraguay

![](img/py/geo/length-scatter.png)

![](img/py/geo/diff-hist.png)

![](img/py/geo/diff-cdf.png)